In [ ]:
import pandas as pd
import seaborn as sns
import plotly.graph_objs as go

# Procesado de los datos

## Lectura de datos

In [2]:
df = pd.read_csv("flights.csv")
pd.set_option('display.max_columns', None)
df.head()

C:\Users\gfine\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Eliminamos las filas duplicadas. Un mismo vuelo no puede retrasarse en el mismo momento del tiempo más de una vez
df = df.drop_duplicates()
df.size

180391449

## Análisis de variables

In [4]:
df.columns

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
       'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5819079 entries, 0 to 5819078
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  int64  
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED             int64  
 24

Columnas que se pueden eliminar:
- TAIL NUMBER: representa un ID de avión único por lo que no corresponde con una variable representativa para el análisis por lo que la eliminamos

### Debatir:
Nos importan..?
- SCHEDULED_DEPARTURE: yo diría que no, si llega en hora = OK!
- DEPARTURE_TIME: es interesante analizar los retrasos por tramo horario?
- DEPARTURE_DELAY: yo diría que no, si sale tarde pero llega en hora = OK!
- ARRIVAL_TIME: yo diría que no, solo me interesa si el vuelo llega tarde, no?
- SCHEDULED_TIME: no se a que se refiere  
- SCHEDULED_ARRIVAL: solo nos interesa saber si se retrasa o no, no la previsión

In [6]:
# Eliminamos tail number porque es un identificador de avión (no vuelo) único
df = df.drop(["TAIL_NUMBER"],axis=1)

Hay un error de formato con la variable flight number, que es un indicador del vuelo y por lo tanto una variable categórica. Lo mismo ocurre con Cancelled y Diverted

In [7]:
# Añadimos el FlightNum, Cancelled and Diverted como variables categóricas
df['FLIGHT_NUMBER']=df['FLIGHT_NUMBER'].astype(object) 
df['CANCELLED']=df['CANCELLED'].astype(object) 
df['DIVERTED']=df['DIVERTED'].astype(object) 

In [8]:
# Unimos las columnas Year, Month y Day of Month como una única variable fecha
# Formato por defecto mes/dia/año para que to_datetime funcione correctamente
df["DATE"]  = df['MONTH'].astype(str) +'/'+ df['DAY'].astype(str) +'/' + df['YEAR'].astype(str)
df["DATE"] = pd.to_datetime(df["DATE"])

Si nos interesa en algún momento podríamos añadir las horas también 

In [9]:
#Eliminamos las columnas year, month y day
df = df.drop(["YEAR","MONTH", "DAY"],axis=1)

# También podemos eliminar la columna DAY_OF_WEEK ya que podemos obtenerla
df = df.drop("DAY_OF_WEEK",axis=1)
print(f"Ejemplo: obtener dia de la semana de la fecha {df['DATE'].iloc[0]} --> {df['DATE'].iloc[0].day_name()} (dia {df['DATE'].iloc[0].dayofweek})")

# NOTA: hay que tener en cuenta que dayofweek empieza a contar en 0 = lunes

Ejemplo: obtener dia de la semana de la fecha 2015-01-01 00:00:00 --> Thursday (dia 3)


In [10]:
df.head()

,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE
0,AS,98,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01
1,AA,2336,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01
2,US,840,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01
3,AA,258,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01
4,AS,135,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01


Hablar formato tiempos: 
    SCHEDULED_DEPARTURE, DEPARTURE_TIME, SCHEDULED_TIME
    SCHEDULED_ARRIVAL, ARRIVAL_TIME (se podrían eliminar dado que nos sirven para sacar arrival delay)

## Debatir:

Las variables SCHEDULED_DEPARTURE, DEPARTURE_TIME, DEPARTURE_DELAY, ARRIVAL_TIME y SCHEDULED_ARRIVAL nos ayudan a saber si un vuelo se ha retrasado o no, pero una vez aue sabemos esto no tienen  valor en sí mismas por lo que las podemos eliminar

#### Tipos de vuelo:
1. On time/ arrived earlier --> arrival_delay <=0
2. Delayed   --> arrival_delay > 0
3. Diverted  --> diverted == 1
4. Cancelled --> cancelled == 1

##### Razones por las que se retrasa un vuelo:
- AIR_SYSTEM_DELAY     
- SECURITY_DELAY       
- AIRLINE_DELAY        
- LATE_AIRCRAFT_DELAY 
- WEATHER_DELAY       

In [11]:
# Vamos a examinar ahora los valores nulos
df.isna().sum()

AIRLINE                      0
FLIGHT_NUMBER                0
ORIGIN_AIRPORT               0
DESTINATION_AIRPORT          0
SCHEDULED_DEPARTURE          0
DEPARTURE_TIME           86153
DEPARTURE_DELAY          86153
TAXI_OUT                 89047
WHEELS_OFF               89047
SCHEDULED_TIME               6
ELAPSED_TIME            105071
AIR_TIME                105071
DISTANCE                     0
WHEELS_ON                92513
TAXI_IN                  92513
SCHEDULED_ARRIVAL            0
ARRIVAL_TIME             92513
ARRIVAL_DELAY           105071
DIVERTED                     0
CANCELLED                    0
CANCELLATION_REASON    5729195
AIR_SYSTEM_DELAY       4755640
SECURITY_DELAY         4755640
AIRLINE_DELAY          4755640
LATE_AIRCRAFT_DELAY    4755640
WEATHER_DELAY          4755640
DATE                         0
dtype: int64

Las variables CANCELLATION_REASON, AIR_SYSTEM_DELAY, SECURITY_DELAY, AIRLINE_DELAY, LATE_AIRCRAFT_DELAY,WEATHER_DELAY presentan una gran cantidad de valores nulos. No obstante, todas estas variables se relacionan con vuelos cancelados o retrasados por lo que tiene sentido que sean valores nulos para aquellos vuelos que no se hayan cancelado ni retrasado. Vamos a analizar estas variables

In [12]:
cancelled = df[df['CANCELLED'] == 1]
cancelled.isna().sum()

AIRLINE                    0
FLIGHT_NUMBER              0
ORIGIN_AIRPORT             0
DESTINATION_AIRPORT        0
SCHEDULED_DEPARTURE        0
DEPARTURE_TIME         86153
DEPARTURE_DELAY        86153
TAXI_OUT               89047
WHEELS_OFF             89047
SCHEDULED_TIME             5
ELAPSED_TIME           89884
AIR_TIME               89884
DISTANCE                   0
WHEELS_ON              89884
TAXI_IN                89884
SCHEDULED_ARRIVAL          0
ARRIVAL_TIME           89884
ARRIVAL_DELAY          89884
DIVERTED                   0
CANCELLED                  0
CANCELLATION_REASON        0
AIR_SYSTEM_DELAY       89884
SECURITY_DELAY         89884
AIRLINE_DELAY          89884
LATE_AIRCRAFT_DELAY    89884
WEATHER_DELAY          89884
DATE                       0
dtype: int64

Vemos como en este caso no existen valores nulos para la columna CANCELLATION_REASON. No obstante, nuestro análisis consiste en prededcir el retraso de vuelos por lo que no necesitamos la info de vuelos cancelados/ redirigidos así que eliminamos dichos registros

In [13]:
# Eliminamos los cancelados
df = df[df['CANCELLED'] == 0]

#Eliminamos los redirigidos
df = df[df['DIVERTED'] == 0]

df = df.drop(["DIVERTED","CANCELLED", "CANCELLATION_REASON"],axis=1)

Nos quedamos solo con los vuelos que llegan antes de lo previsto, en hora o con retraso

In [15]:
len(df)

5714008

In [14]:
df.isna().sum()

AIRLINE                      0
FLIGHT_NUMBER                0
ORIGIN_AIRPORT               0
DESTINATION_AIRPORT          0
SCHEDULED_DEPARTURE          0
DEPARTURE_TIME               0
DEPARTURE_DELAY              0
TAXI_OUT                     0
WHEELS_OFF                   0
SCHEDULED_TIME               0
ELAPSED_TIME                 0
AIR_TIME                     0
DISTANCE                     0
WHEELS_ON                    0
TAXI_IN                      0
SCHEDULED_ARRIVAL            0
ARRIVAL_TIME                 0
ARRIVAL_DELAY                0
AIR_SYSTEM_DELAY       4650569
SECURITY_DELAY         4650569
AIRLINE_DELAY          4650569
LATE_AIRCRAFT_DELAY    4650569
WEATHER_DELAY          4650569
DATE                         0
dtype: int64

Realizamos ahora el análisis de los valores nulos relacionados con los vuelos retrasados.

**NOTA**: consideramos que un vuelo se retrasa si llega pasada la hora prevista, independientemente de si ha tenido retraso en la hora de salida o no

In [16]:
delayed = df[df['ARRIVAL_DELAY'] > 0]
delayed.isna().sum()

AIRLINE                      0
FLIGHT_NUMBER                0
ORIGIN_AIRPORT               0
DESTINATION_AIRPORT          0
SCHEDULED_DEPARTURE          0
DEPARTURE_TIME               0
DEPARTURE_DELAY              0
TAXI_OUT                     0
WHEELS_OFF                   0
SCHEDULED_TIME               0
ELAPSED_TIME                 0
AIR_TIME                     0
DISTANCE                     0
WHEELS_ON                    0
TAXI_IN                      0
SCHEDULED_ARRIVAL            0
ARRIVAL_TIME                 0
ARRIVAL_DELAY                0
AIR_SYSTEM_DELAY       1023457
SECURITY_DELAY         1023457
AIRLINE_DELAY          1023457
LATE_AIRCRAFT_DELAY    1023457
WEATHER_DELAY          1023457
DATE                         0
dtype: int64

Si considerabamos todos los vuelos teníamos 4650569 registros con valor nulo en las variables DELAY, cuando filtramos por vuelos retrasados tenemos 1023457.
Vamos a analizar primero los valores nulos de aquellos vuelos que se han retrasado

In [17]:
delayed = delayed[['AIR_SYSTEM_DELAY', 'SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY']]
delayed.head()

,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
2,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN
21,NaN,NaN,NaN,NaN,NaN


In [18]:
# Seleccionamos todas las filas que tengan NaN en todas las columnas
nulls = delayed.loc[(delayed['AIR_SYSTEM_DELAY'].isnull() == True) & (delayed['SECURITY_DELAY'].isnull() == True) & (delayed['AIRLINE_DELAY'].isnull() == True) & (delayed['LATE_AIRCRAFT_DELAY'].isnull() == True) & (delayed['WEATHER_DELAY'].isnull() == True)]
len(nulls)

1023457

Vemos que TODOS los NaN se concentran en las mismas filas. Entendemos que en este caso el vuelo se ha retrasado por causa desconocida. Para indicar esto creamos una nueva columna 'OTHER_DELAY' en nuestro data frame cuyo valor sea igual al delay

In [ ]:
#pd.isna(df['AIR_SYSTEM_DELAY'].iloc[20])

In [ ]:
# df = df.drop(['OTHER_DELAY'], axis = 1)

In [ ]:
# Creamos la columna vacía
df['OTHER_DELAY'] = pd.Series() 

# Como hemos visto que si una columna es NaN el resto también, utilizamos una única columa para comparar
for i in range(0,len(df),1):
    if (df['ARRIVAL_DELAY'].iloc[i]>0 & pd.isna(df['AIR_SYSTEM_DELAY'].iloc[i])):
        df['OTHER_DELAY'].iloc[i] = df['ARRIVAL_DELAY'].iloc[i]
    else:
        df['OTHER_DELAY'].iloc[i] = 0

C:\Users\gfine\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.
C:\Users\gfine\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
# Early arrival flights
early_arrival = df[df['ARRIVAL_DELAY']<=0]

print(len(early_arrival))
early_arrival.isna().sum()

El resto de valores NaN en dichas columnas corresponden a aquellos vuelos que han llegado antes de tiempo a destino, cosa que tiene sentido dado que no han experimentado ningún delay

In [ ]:
df.isna().sum()

Una vez analizado el por qué de los valores NaN presentes en el dataset parece razonable sustituir dichos valores por 0

In [ ]:
# Sustituimos los valores NaN por ceros
df = df.fillna(0)

fillna() sustituye los NaN de todo el dataframe por lo que es importante destacar que podemos utilizar este método dado que los únicos campos NaN del dataframe se encuentran en las variables de delay analizadas. 

In [ ]:
df.isna().sum()

Resulta interesante saber si los vuelos pueden retrasarse por un único motivo o exclusivamente por uno. Vamos a investigarlo

In [ ]:
# Tomamos la variable WEATHER_DELAY como referencia
weather = df.loc[(df['WEATHER_DELAY'] != df['ARRIVAL_DELAY']) & (df['WEATHER_DELAY']>0 )]
weather.head()

Confirmamos, ¡el retraso puedes estar asociado a varios motivos!

**CHECKPOINT** Datos de vuelos limpios

## Info Aerolíneas

In [ ]:
# Vamos a añadir la el nombre asociado a las airlines
# Cargamos los datos
airlines = pd.read_csv("airlines.csv")
pd.set_option('display.max_columns', None)
airlines.head()

In [ ]:
# Renombramos la columna "AIRLINE" para poder hacer el join con la tabla de aerolineas
df = df.rename(columns={"AIRLINE": "IATA_CODE"})
df.head()

In [ ]:
# Unimos ambas tablas
flights = df.merge(airlines, on='IATA_CODE', how='left')

## Info Aeropuertos

In [ ]:
# Vamos a añadir info relativa a los aeropuertos
# Cargamos los datos
airports= pd.read_csv("airports.csv")
pd.set_option('display.max_columns', None)
airports.head()

In [ ]:
# Guardamos los datos preprocesados, para ser utilizados en la predicción posterior
flights.to_csv("flightsCleaned.csv", index=False)

# EDA

In [ ]:
# Variables categoricas
flights.select_dtypes('object').nunique()

In [ ]:
# Variables numéricas
flights.describe()

In [ ]:
# Correlación entre variables númericamente
corr = flights.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(4)

NOTA: cambiar colores :)

In [ ]:
# Habría que prescindir de las variables con correlación alta dado que aportan información redundante

In [ ]:
# Faltan plots para distribuciones de variables categóricas y numéricas